<div style="text-align: right">INFO 6105 Data Science Eng Methods and Tools, assignemnt 2</div>
<div style="text-align: right">Zixiao Wang, 14 September 2019</div>

# Genetic sort and plant growth drawing

## Goals

1. Build a genetic sort algorithm to sort a list of integers
2. Compare the genetic algorithm performance with bubble sort and merge sort
3. Draw the fern that create with "plant_growth(6,output="X", show = Flase)"

---

## Goal 1 Genetic sort

The sort algorithm use random integert list as **genes set**. Because we don't know what the sorted list look like. So I will use "if the list is sorted" as **target**. The **fitness function** will give the score base on whether the first few places are sorted or not. This means that the function determines whether the number of the current position is the smallest of all the numbers that follow.

### Random list generate function

This function can generate random integert list base the input length and range

In [1]:
import random
def random_int(length,num_range):
    return random.choices(range(num_range),k=length)

In [2]:
random_int(10,1000)

[996, 493, 677, 507, 251, 526, 632, 809, 12, 92]

### Genes

Use the random integert list as geneSet

In [3]:
random.seed(4)
geneSet = random_int(10,100)
target = False

print(geneSet)

[23, 10, 39, 15, 6, 40, 91, 80, 76, 22]


## Gnerate a guess

In [4]:
def generate_parent(length):
    return random.sample(geneSet,length)
    

---
#### TEST

In [5]:
test = []
test = generate_parent(len(geneSet))
print(test)

[76, 40, 6, 91, 10, 23, 39, 80, 15, 22]


---

## Fitness
The fitness function will give the score base on whether the first few places are sorted or not.

In [6]:
def get_fitness(guess):
    return sum(1 for i in range(len(guess)) if guess[i]==min(guess[i:]))

---
#### TEST

In [7]:
test = [2,34,1,5,4]
print(min(test[3:]))
print(test[4])

4
4


In [8]:
print(get_fitness(test))
print(get_fitness([1,2,3]))
print(get_fitness([3,2,1]))

2
3
1


---

## Mutate

The function exchange random 2 number from parent genes as mutate

In [9]:
def mutate(parent):
    index1,index2 = random.sample(range(len(parent)),2)
    parent[index1],parent[index2] = parent[index2],parent[index1]
    childGenes = parent
    return childGenes

---
#### TEST

In [10]:
test = [123,1,231,24,1,23,12,3,1,41,42,2]
random.sample(range(len(test)),2)

[4, 3]

In [11]:
test = [1,2]
test[0],test[1] = test[1],test[0]
test

[2, 1]

In [12]:
test = [1,2,3]

In [13]:
test.remove(3)
test

[1, 2]

---

## Display

In [14]:
import datetime

def display(guess,startTime):
    timeDiff = datetime.datetime.now()-startTime
    fitness = get_fitness(guess)
    print("{0}\t  {1}\t{2}".format(guess,fitness,str(timeDiff)))

## Main

Initializing parent

In [15]:
random.seed()
startTime = datetime.datetime.now()
parent = generate_parent(len(geneSet))
parentFitness = get_fitness(parent)
display(parent,startTime)

[23, 80, 6, 10, 40, 39, 91, 15, 22, 76]	  5	0:00:00


Loop for sorting

In [16]:
%%time
while True:
    child = mutate(parent)
    childFitness = get_fitness(child)
    
    if parentFitness >= childFitness:
        continue
    display(child,startTime)
    if childFitness >= len(geneSet):
        break
    parentFitness = childFitness
    parent = child

[6, 10, 22, 39, 15, 23, 80, 40, 91, 76]	  6	0:00:00.092751
[91, 6, 10, 15, 76, 22, 80, 23, 39, 40]	  7	0:00:00.099201
[6, 10, 15, 22, 76, 40, 23, 39, 80, 91]	  8	0:00:00.126977
[6, 91, 10, 15, 22, 23, 39, 40, 76, 80]	  9	0:00:03.721492
[6, 10, 15, 22, 23, 39, 40, 76, 80, 91]	  10	0:01:05.284059
Wall time: 1min 5s


---

## Goal 2 Compare

### Bubble sort

In [17]:
def bubble_sort(alist):
    for passnum in range(len(alist)-1, 0, -1):
        for i in range(passnum):
            if alist[i] > alist[i+1]:
                temp = alist[i]
                alist[i] = alist[i+1]
                alist[i+1] = temp

In [18]:
%%time
bubble_sort(geneSet)

Wall time: 0 ns


### Merge sort

In [19]:
def merge_sort(alist):
    #print("Splitting ", alist)
    
    if len(alist) > 1:
        mid = len(alist)//2
        lefthalf = alist[:mid]
        righthalf = alist[mid:]
        
        merge_sort(lefthalf)
        merge_sort(righthalf)
        
        i = 0
        j = 0
        k = 0
        
        while i < len(lefthalf) and j < len(righthalf):
            if lefthalf[i] < righthalf[j]:
                alist[k] = lefthalf[i]
                i += 1
            else:
                alist[k] = righthalf[j]
                j += 1
                
            k += 1
                
        while i < len(lefthalf):
            alist[k] = lefthalf[i]
            i += 1
            k += 1
            
        while j < len(righthalf):
            alist[k] = righthalf[j]
            j += 1
            k += 1

In [20]:
%%time
merge_sort(geneSet)

Wall time: 0 ns


---

## Goal 3 The fern drawing

### Plant growth function

In [21]:
def plant_growth(number, output="X", show=False):
    for i in range(number):
        newOutput = ""
        for letter in output:
            # rule #1
            if letter == "X":
                newOutput += "F-[[X]+X]+F[+FX]-X"
            elif letter == "F":
                newOutput += "FF"
            else:
                newOutput += letter
        output = newOutput
        if show: print("n =", i+1, output, "[", len(output), "]")
    return output

### Plant drawing class

* ```F``` = go forward
* ```X``` = stay
* ```+``` = turn right 25 degrees
* ```-``` = turn left 25 degrees
* ```[``` = save current (x, y) position
* ```]``` = go back to saved (x, y) position

#### Plant drawing class

In [22]:
from turtle import Turtle
import random

class PlantDrawing(object):
    savedPosition = []
    myTurtle = Turtle()
    plantGrowth = ""
    
    def __init__(self,plantGrowth):
        # init the plant_growth attribute
        self.plantGrowth = plantGrowth
        self.myTurtle.speed(10)
        self.myTurtle.hideturtle()
        self.myTurtle.left(90)
        self.myTurtle.penup()
        self.myTurtle.goto(0,-500)
        self.myTurtle.pendown()
        
    def action(self,command):
        
        if command == 'F':
            self.myTurtle.color(random.choice(["blue","red","green"]))
            self.myTurtle.fd(10)
        elif command == '+':
            self.myTurtle.right(25)
        elif command == '-':
            self.myTurtle.left(25)
        elif command == '[':
            self.savedPosition=self.myTurtle.pos()
        elif command == ']':
            self.myTurtle.penup()
            self.myTurtle.goto(self.savedPosition)
            self.myTurtle.pendown()
        
    def draw_plant_growth(self):
        # draw the plant growth
        for command in list(self.plantGrowth):
            self.action(command)
        

#### Draw the plant

In [23]:
plant=plant_growth(6, output="X", show=False)
draw = PlantDrawing(plantGrowth = plant)
draw.draw_plant_growth()

Terminator: 